Download GNPS data <br>
Replace Job ID below with your GNPS job ID:

In [1]:
! curl -d "" "https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=b76dd5a123e54a7eb42765499f9163a5&view=download_cytoscape_data" -o GNPS_output_graphML_Classical.zip

from zipfile import ZipFile
zf = ZipFile("GNPS_output_graphML_Classical.zip", 'r')
zf.extractall("GNPS_output_graphML_Classical")
zf.close()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  437k    0  437k    0     0   437k      0 --:--:-- --:--:-- --:--:--  999k


In [2]:
! curl -d "" "https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=4d971b8162644e869a68faa35f01b915&view=view_significant" -o Varquest_output.zip

from zipfile import ZipFile
zf = ZipFile("Varquest_output.zip", 'r')
zf.extractall("Varquest_output")
zf.close()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 30644    0 30644    0     0  30644      0 --:--:-- --:--:-- --:--:--  127k


Download <i>in silico</i> structural annotation (Dereplicator, Varquest) data <br>
Replace Job IDs below with your Dereplicator or Varquest job IDs:

In [3]:
! curl -d "" "https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=c62d3283752f4f98b1720d0a6d1ee65b&view=view_significant" -o Derep_output.zip

from zipfile import ZipFile
zf = ZipFile("Derep_output.zip", 'r')
zf.extractall("Derep_output")
zf.close()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1348  100  1348    0     0   1348      0  0:00:01 --:--:--  0:00:01  8585


specify your NAP job ID:

In [4]:
nap_id = 'cb63770fe307410492468f62f9edb8f3'

load libraries

In [5]:
from pyMolNetEnhancer import *
import pandas as pd
import os
import csv 
import json
from networkx import *
path = 'C:\Program Files\ChemAxon\MarvinSuite\bin' #path to MarvinSuite's molconvert used to convert SMILES to InchiKeys
os.environ['PATH'] += ':'+path

In [6]:
if 'clusterinfo_summary' in os.listdir('GNPS_output_graphML_Classical/') and 'DB_result' in os.listdir('GNPS_output_graphML_Classical/'):
    netfile = 'GNPS_output_graphML_Classical/clusterinfo_summary/' + str(os.listdir('GNPS_output_graphML_Classical/clusterinfo_summary/')[0]) 
    gnpslibfile = 'GNPS_output_graphML_Classical/DB_result/'+ str(os.listdir('GNPS_output_graphML_Classical/DB_result/')[0]) 
elif 'clusterinfosummarygroup_attributes_withIDs_withcomponentID' in os.listdir('GNPS_output_graphML_Classical/'):
    netfile = 'GNPS_output_graphML_Classical/clusterinfosummarygroup_attributes_withIDs_withcomponentID/' + str(os.listdir('GNPS_output_graphML_Classical/clusterinfosummarygroup_attributes_withIDs_withcomponentID/')[0])
    gnpslibfile = 'GNPS_output_graphML_Classical/result_specnets_DB/'+ str(os.listdir('GNPS_output_graphML_Classical/result_specnets_DB/')[0])
else:
    netfile = 'GNPS_output_graphML_Classical/clusterinfosummary/' + str(os.listdir('GNPS_output_graphML_Classical/clusterinfosummary/')[0])
    gnpslibfile = 'GNPS_output_graphML_Classical/result_specnets_DB/'+ str(os.listdir('GNPS_output_graphML_Classical/result_specnets_DB/')[0]) 

In [7]:
nap = pd.read_csv("http://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task="+nap_id+"&block=main&file=final_out/node_attributes_table.tsv", sep = "\t")
gnpslib = pd.read_csv(gnpslibfile, sep='\t')
derep = pd.read_csv('Derep_output/'+[s for s in os.listdir('Derep_output') if "DEREPLICATOR" in s][0], sep = '\t')
varquest = pd.read_csv('Varquest_output/'+[s for s in os.listdir('Varquest_output') if "DEREPLICATOR" in s][0], sep = '\t')

add all chemical structural information output as dataframe items in list

In [8]:
matches = [gnpslib, nap, derep, varquest]

In [9]:
out = unique_smiles(matches)

In [10]:
out['df'].to_csv("SMILES.csv", quoting=csv.QUOTE_NONE, escapechar='&')

convert SMILES to InchiKeys

In [11]:
! molconvert inchikey:SAbs SMILES.csv{csv:strucSMILES} -o InchiKeys_Classical.txt

'molconvert' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
ikeys = pd.read_csv("InchiKeys_Classical.txt",  sep='\t',header = None)

out['df']["inchikey"] = ikeys
inchi_dic = make_inchidic(out)

ikeys.columns = ["InChIKey"]
ikeys.to_csv("InchiKeys_Classical.txt", quoting=csv.QUOTE_NONE, escapechar='&')

FileNotFoundError: File b'InchiKeys_Classical.txt' does not exist

retrieve ClassyFire classifcations

In [ ]:
get_classifications("InchiKeys_Classical.txt")

In [ ]:
with open("all_json.json") as tweetfile:
    jsondic = json.loads(tweetfile.read())


df = make_classy_table(jsondic)
df = df.rename(columns = {'class':'CF_class','smiles':'SMILES'})
net = pd.read_csv(netfile,  sep='\t')


final = molfam_classes(net,df,inchi_dic)

In [ ]:
final.head()

write ClassyFire results per molecular family to file. To visualize chemical classes within the mass spectral molecular network, this file can be imported as table into Cytoscape.

In [ ]:
final.to_csv("ClassyFireResults_Network_Classical.txt", sep = '\t', index = False)

how many InChIKeys could not be classified? (percentage of total submitted InChIKeys)

In [ ]:
len(set(list(ikeys.InChIKey)) - set(list(df.inchikey)))/len(set(list(ikeys.InChIKey)))

which InChIKeys could not be classified?

In [ ]:
set(list(ikeys.InChIKey)) - set(list(df.inchikey))

create graphML file

In [ ]:
if any("FEATURE" in s for s in os.listdir('GNPS_output_graphML_Classical/')):
    graphMLfile = 'GNPS_output_graphML_Classical/' + [x for x in os.listdir('GNPS_output_graphML_Classical/') if 'FEATURE' in x][0]
    graphML = read_graphml(graphMLfile)
    graphML_classy = make_classyfire_graphml(graphML,final)
    nx.write_graphml(graphML_classy, "ClassyFireResults_Network.graphml", infer_numeric_types = True)
elif any("METABOLOMICS" in s for s in os.listdir('GNPS_output_graphML_Classical/')):
    graphMLfile = 'GNPS_output_graphML_Classical/' + [x for x in os.listdir('GNPS_output_graphML_Classical/') if 'METABOLOMICS' in x][0]
    graphML = read_graphml(graphMLfile)
    graphML_classy = make_classyfire_graphml(graphML,final)
    nx.write_graphml(graphML_classy, "ClassyFireResults_Network_Classical.graphml", infer_numeric_types = True)
else:
    print('There is no graphML file for this GNPS molecular network job')